In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow.keras as keras

In [3]:
from keras import layers, Sequential
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [4]:
import os

In [5]:
import pickle

In [6]:
os.getcwd()

'/content'

In [7]:
os.listdir(os.getcwd())

['.config', 'drive', 'sample_data']

In [8]:
pickle_file= os.getcwd()+'/drive/MyDrive/Sub_Data/data.pickle'

In [9]:
pickle_file

'/content/drive/MyDrive/Sub_Data/data.pickle'

In [10]:
df = pd.read_pickle(pickle_file)

In [11]:
df

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,Labels
0,-358.717529,-6.362634,-18.675270,2.614578,6.788164,-16.964695,19.294222,3.831350,-1.007537,17.229862,1.682296,-10.763758,-7.563745,blues
1,-373.422455,28.541172,-62.292374,33.772362,-24.182438,24.464520,14.590254,10.334937,-33.561497,25.110397,-2.553226,-12.549841,-11.501458,blues
2,-379.265686,37.311401,-62.880608,33.969246,-15.152157,23.453659,3.502853,1.759964,-27.262371,18.922270,-4.817671,-28.283806,1.101586,blues
3,-394.471283,39.922462,-71.621735,41.641197,-6.377332,24.744980,20.557190,4.765609,-24.279747,2.292906,-9.145906,-16.167883,1.773674,blues
4,-405.742004,53.017689,-49.039005,26.963558,-19.102318,22.194271,13.690291,8.213572,-58.082420,7.680556,3.642120,-24.813526,5.854433,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298175,-637.151001,34.008850,-59.231636,33.153923,-13.896715,1.681734,-8.170829,-0.926614,-13.010382,26.459517,4.632920,7.670885,-14.734737,classical
1298176,-635.690308,35.429245,-69.809265,32.217850,-6.378393,-6.269723,-17.840149,4.386957,2.011418,36.566162,7.126342,-3.887561,-17.075964,classical
1298177,-643.668945,26.324226,-72.707924,18.648598,-8.029517,11.763157,-15.609495,2.920358,-4.031073,25.876705,2.650877,1.302766,-19.647341,classical
1298178,-658.121094,19.917147,-63.834007,23.408195,-12.162072,11.127884,-14.975857,0.325112,-10.495857,12.072266,0.321636,8.379921,-13.380528,classical


In [12]:
#calculations of dataframe

#n_mfcc = 13,
#n_fft = 248,
#hop_length = 512, num_segment = 10
#sample_rate = 22050,
#track_duration = 30,
#samples_per_track = sample_rate * track_duration = 661500
#num_samples_per_segment = int(samples_per_track/num_segments) = 66150
#expected_num_mfcc_vectors_per_segment = math.ceil(num__samples_per_segment/hop_length)=130

In [13]:
input = df.iloc[0*130:0+1*130,:-1]
input

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13
0,-358.717529,-6.362634,-18.675270,2.614578,6.788164,-16.964695,19.294222,3.831350,-1.007537,17.229862,1.682296,-10.763758,-7.563745
1,-373.422455,28.541172,-62.292374,33.772362,-24.182438,24.464520,14.590254,10.334937,-33.561497,25.110397,-2.553226,-12.549841,-11.501458
2,-379.265686,37.311401,-62.880608,33.969246,-15.152157,23.453659,3.502853,1.759964,-27.262371,18.922270,-4.817671,-28.283806,1.101586
3,-394.471283,39.922462,-71.621735,41.641197,-6.377332,24.744980,20.557190,4.765609,-24.279747,2.292906,-9.145906,-16.167883,1.773674
4,-405.742004,53.017689,-49.039005,26.963558,-19.102318,22.194271,13.690291,8.213572,-58.082420,7.680556,3.642120,-24.813526,5.854433
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,-553.638550,115.547859,-11.745089,3.652759,12.958879,7.425889,-12.086588,-12.838453,6.056415,0.925507,-8.802722,4.830578,8.481441
126,-515.812256,81.606880,-30.202282,26.742224,0.390943,4.433127,-0.263771,3.414610,3.164185,6.794114,-8.196673,-8.922878,9.851894
127,-461.981842,63.263069,-41.111145,31.014582,-5.264947,1.090261,-12.309715,10.950979,-11.831028,7.117218,-17.210554,-15.731875,-13.295950
128,-484.115234,85.535843,-35.242767,35.987541,3.471575,3.514775,0.937387,0.554750,-16.195789,-1.557993,-2.944598,1.448021,-14.577988


In [14]:
print(f"Shape of input: {input.shape}")

Shape of input: (130, 13)


In [15]:
input = np.expand_dims(input,0) #expanded the shape of the array by adding a new axis at the 0th position 
input.shape

(1, 130, 13)

In [16]:
target= [df.iloc[0*130,-1]]
target

['blues']

In [17]:
def process_df(df):
    '''
    Input: dataframe object
    Output: returns a 3d nparray
    '''
    inputs = df.iloc[0*130:0+1*130,:-1] #getting the first 130 rows
    inputs = np.expand_dims(inputs,0)   #expanding the shape of the array by inserting a new axis at the first position
    targets= [df.iloc[0*130,-1]]
    for i in range(1, int(df.shape[0]/130)): #repeating the above process for the entirety of the remaining dataset
        dummy_x = df.iloc[i*130:(i+1)*130, :-1].values
        dummy_x = np.expand_dims(dummy_x,0)
        dummy_y = df.iloc[i*130,-1]
        inputs = np.concatenate((inputs, dummy_x), axis=0)
        targets.append(dummy_y)
    return inputs, targets

In [18]:
inputs, targets = process_df(df)

In [19]:
inputs.shape

(9986, 130, 13)

In [20]:
len(targets)

9986

In [21]:
type(inputs)

numpy.ndarray

In [22]:
inputs.dtype

dtype('float32')

In [23]:
inputs

array([[[-3.58717529e+02, -6.36263418e+00, -1.86752701e+01, ...,
          1.68229556e+00, -1.07637577e+01, -7.56374454e+00],
        [-3.73422455e+02,  2.85411720e+01, -6.22923737e+01, ...,
         -2.55322552e+00, -1.25498409e+01, -1.15014582e+01],
        [-3.79265686e+02,  3.73114014e+01, -6.28806076e+01, ...,
         -4.81767130e+00, -2.82838058e+01,  1.10158634e+00],
        ...,
        [-4.61981842e+02,  6.32630692e+01, -4.11111450e+01, ...,
         -1.72105541e+01, -1.57318754e+01, -1.32959499e+01],
        [-4.84115234e+02,  8.55358429e+01, -3.52427673e+01, ...,
         -2.94459820e+00,  1.44802141e+00, -1.45779877e+01],
        [-4.82298950e+02,  9.57239151e+01, -4.68237495e+01, ...,
         -1.46130953e+01,  1.85669422e+00, -1.87632217e+01]],

       [[-4.98308105e+02,  9.13460999e+01, -3.89359055e+01, ...,
         -1.63229694e+01,  1.06008034e+01, -6.07426119e+00],
        [-4.69089386e+02,  6.82946014e+01, -3.39981308e+01, ...,
         -1.27064991e+01, -4.01786232e

In [24]:
#encoding categorical values 
encoder = LabelEncoder()
targets = encoder.fit_transform(targets) #encoded the categorical values in our target class 

In [25]:
targets

array([0, 0, 0, ..., 1, 1, 1])

In [47]:
np.unique(targets)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [49]:
list(encoder.inverse_transform([0,1,2,3,4,5,6,7,8,9]))

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [26]:
#Splitting data into train and test sets
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs, targets, test_size = 0.3)

In [27]:
inputs.shape

(9986, 130, 13)

In [28]:
inputs_train.shape

(6990, 130, 13)

In [29]:
inputs_test.shape

(2996, 130, 13)

In [30]:
#Building a neural network model
model = keras.Sequential([
    #inpute layer
    keras.layers.Flatten(input_shape = (inputs_train.shape[1], inputs_train.shape[2])),

    #1st dense layer
    keras.layers.Dense(512, activation = 'relu'),

    #2nd dense layeer
    keras.layers.Dense(256, activation = 'relu'),

    #3rd dense layer
    keras.layers.Dense(64, activation = 'relu'),

    #output layer
    keras.layers.Dense(10,activation ='softmax')
])

In [31]:
#compile model
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer = optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1690)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               865792    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 1,014,218
Trainable params: 1,014,218
Non-trainable params: 0
_________________________________________________________________


In [33]:
#train the network
nn_model = model.fit(inputs_train, targets_train,
                    validation_data = (inputs_test, targets_test),
                    epochs=50,
                    batch_size=32)

Epoch 1/50
219/219 [==============================] - 2s 8ms/step - loss: 14.4761 - accuracy: 0.2711 - val_loss: 8.3711 - val_accuracy: 0.2840
Epoch 2/50
219/219 [==============================] - 2s 7ms/step - loss: 6.5103 - accuracy: 0.3435 - val_loss: 5.3536 - val_accuracy: 0.3501
Epoch 3/50
219/219 [==============================] - 2s 8ms/step - loss: 4.8588 - accuracy: 0.3890 - val_loss: 7.0593 - val_accuracy: 0.2984
Epoch 4/50
219/219 [==============================] - 2s 7ms/step - loss: 4.1761 - accuracy: 0.4129 - val_loss: 4.8504 - val_accuracy: 0.4049
Epoch 5/50
219/219 [==============================] - 2s 7ms/step - loss: 3.6600 - accuracy: 0.4335 - val_loss: 3.8187 - val_accuracy: 0.4219
Epoch 6/50
219/219 [==============================] - 2s 8ms/step - loss: 3.1713 - accuracy: 0.4690 - val_loss: 4.2549 - val_accuracy: 0.3985
Epoch 7/50
219/219 [==============================] - 2s 7ms/step - loss: 2.6508 - accuracy: 0.5099 - val_loss: 3.4576 - val_accuracy: 0.4229
Epoch

# Saving a model in pickel file

In [34]:
import pickle
import os

In [35]:
os.getcwd()

'/content'

In [36]:
os.listdir(os.getcwd())

['.config', 'drive', 'sample_data']

In [37]:
model_path = os.path.join(os.getcwd(), 'drive/Sub_Data')

In [38]:
model_path

'/content/drive/Sub_Data'

In [42]:
model.save('nn_model.h5')